# Hidden Layer Quickstart

This notebook verifies your environment and demonstrates the core harness APIs.
Feel free to edit and save under a new name once everything is working.

In [ ]:
import os
import shutil
from datetime import datetime

from harness import ExperimentConfig, ExperimentResult, get_tracker, llm_call, run_strategy
from harness.defaults import DEFAULT_MODEL, DEFAULT_PROVIDER

providers = []
if shutil.which("ollama"):
    providers.append("ollama")
try:
    import mlx.core  # noqa: F401
    providers.append("mlx")
except Exception:
    pass
if os.getenv("ANTHROPIC_API_KEY"):
    providers.append("anthropic")
if os.getenv("OPENAI_API_KEY"):
    providers.append("openai")

if not providers:
    providers.append(DEFAULT_PROVIDER)

print("Detected providers:", providers)
print("Default model:", DEFAULT_MODEL)
selected_provider = providers[0]
print("Using provider for smoke test:", selected_provider)

In [ ]:
try:
    response = llm_call("Say 'Hidden Layer online.'", provider=selected_provider, model=None)
    print("Model:", response.model)
    print("Latency (s):", f"{response.latency_s:.2f}")
    print("Response:", response.text.strip())
except Exception as exc:
    raise RuntimeError("LLM smoke test failed. Check provider configuration.") from exc

In [ ]:
strategy_result = run_strategy(
            "single",
            "List two reasons multi-agent strategies can outperform single models.",
            provider=selected_provider,
            model=None,
            verbose=False,
        )
        selected_model = strategy_result.metadata.get("model") if strategy_result.metadata else DEFAULT_MODEL
        print("Strategy:", strategy_result.strategy_name)
        print("Latency (s):", f"{strategy_result.latency_s:.2f}")
        print("Output:
", strategy_result.output.strip())

In [ ]:
config = ExperimentConfig(
    experiment_name="quickstart_smoke",
    task_type="demo",
    strategy="single",
    provider=selected_provider,
    model=selected_model,
)
tracker = get_tracker()
run_dir = tracker.start_experiment(config)

result_record = ExperimentResult(
    config=config,
    task_input="Quickstart notebook demo task",
    output=strategy_result.output,
    latency_s=strategy_result.latency_s,
    tokens_in=strategy_result.tokens_in,
    tokens_out=strategy_result.tokens_out,
    cost_usd=strategy_result.cost_usd,
    eval_scores={},
    eval_metadata={},
    success=True,
    error=None,
)
tracker.log_result(result_record)
summary = tracker.finish_experiment()
if summary:
    print("Logged experiment in:", run_dir)
    print("Summary runs:", summary.get("total_runs"))
    print("Average latency (s):", f"{(summary.get('avg_latency_s') or 0):.2f}")
    print("Timestamp:", datetime.fromisoformat(summary["timestamp"]))
else:
    print("No summary generated (no results logged).")